# 调用本地ollama模型

In [ ]:
from langchain_ollama import ChatOllama

llm = ChatOllama(
    model="gemma3:latest",
    temperature=0,
    # other params...
)
messages = [
    (
        "system",
        "You are a helpful assistant that translates English to chinese. Translate the user sentence.",
    ),
    ("human", "I love programming."),
]
ai_msg = llm.invoke(messages)
print(ai_msg.content)



# 官方示例调用查询天气工具agent:
配置模型api坑：使用os.enviro和getpass动态获取 google api key会失败（os.environ 是在 当前 Python 进程的内存中生效的，而 LangChain 在内部读取 API key 的方式可能在实例化 ChatGoogleGenerativeAI 前已经缓存了环境变量状态。）

In [ ]:
from dataclasses import dataclass
from langchain.agents import create_agent
from langchain.tools import tool, ToolRuntime
from langgraph.checkpoint.memory import InMemorySaver
from langchain.agents.structured_output import ToolStrategy
from langchain_google_genai import ChatGoogleGenerativeAI



# Define system prompt
SYSTEM_PROMPT = """You are an expert weather forecaster, who speaks in puns.

You have access to two tools:

- get_weather_for_location: use this to get the weather for a specific location
- get_user_location: use this to get the user's location

If a user asks you for the weather, make sure you know the location. If you can tell from the question that they mean wherever they are, use the get_user_location tool to find their location."""

# Define context schema
@dataclass
class Context:
    """Custom runtime context schema."""
    user_id: str

# Define tools
@tool
def get_weather_for_location(city: str) -> str:
    """Get weather for a given city."""
    return f"It's always sunny in {city}!"

@tool
def get_user_location(runtime: ToolRuntime[Context]) -> str:
    """Retrieve user information based on user ID."""
    user_id = runtime.context.user_id
    return "Florida" if user_id == "1" else "SF"

# Configure claude model
# model = init_chat_model(
#     "claude-sonnet-4-5-20250929",
#     temperature=0
# )

model = ChatGoogleGenerativeAI(
    model="gemini-2.5-flash",
    temperature=1.0,  
    max_tokens=None,
    timeout=None,
    max_retries=2,
)


# Define response format
@dataclass
class ResponseFormat:
    """Response schema for the agent."""
    # A punny response (always required)
    punny_response: str
    # Any interesting information about the weather if available
    weather_conditions: str | None = None

# Set up memory
checkpointer = InMemorySaver()

# Create agent
agent = create_agent(
    model=model,
    system_prompt=SYSTEM_PROMPT,
    tools=[get_user_location, get_weather_for_location],
    context_schema=Context,
    response_format=ToolStrategy(ResponseFormat),
    checkpointer=checkpointer
)

# Run agent
# `thread_id` is a unique identifier for a given conversation.
config = {"configurable": {"thread_id": "1"}}

response = agent.invoke(
    {"messages": [{"role": "user", "content": "what is the weather outside?"}]},
    config=config,
    context=Context(user_id="1")
)

print(response['structured_response'])
# ResponseFormat(
#     punny_response="Florida is still having a 'sun-derful' day! The sunshine is playing 'ray-dio' hits all day long! I'd say it's the perfect weather for some 'solar-bration'! If you were hoping for rain, I'm afraid that idea is all 'washed up' - the forecast remains 'clear-ly' brilliant!",
#     weather_conditions="It's always sunny in Florida!"
# )


# Note that we can continue the conversation using the same `thread_id`.
response = agent.invoke(
    {"messages": [{"role": "user", "content": "thank you!"}]},
    config=config,
    context=Context(user_id="1")
)

print(response['structured_response'])
# ResponseFormat(
#     punny_response="You're 'thund-erfully' welcome! It's always a 'breeze' to help you stay 'current' with the weather. I'm just 'cloud'-ing around waiting to 'shower' you with more forecasts whenever you need them. Have a 'sun-sational' day in the Florida sunshine!",
#     weather_conditions=None
# )

ResponseFormat(punny_response="Looks like you're in for a re-sun-able day! Don't get too ex-straw-dinary with your outdoor activities!", weather_conditions='Sunny')
ResponseFormat(punny_response="You're welcome! I'm always here to help you weather any storm of questions!", weather_conditions=None)


# Build a rag agent with langchain
1. 设置api_key环境变量

In [8]:
import getpass
import os

def _set_env(key: str, model_name: str):
    if key not in os.environ:
        os.environ[key] = getpass.getpass(f"{key}:")
        print("已在当前notebook kerner里设置临时环境变量[GOOGLE_API_KEY],重启kernel后失效")
    else:
        print("API_KEY前五位:"+ os.environ["GOOGLE_API_KEY"][:5])  # 只看前5位，避免泄露
        # 只有在环境变量已存在时才调用 Google 模型
        from langchain_google_genai import ChatGoogleGenerativeAI #延迟导入，确认环境变量已经存在，notebook kernel长时间不重启环境变量可能存在问题，推荐延迟导入

        llm = ChatGoogleGenerativeAI(
            model=model_name,
            temperature=0.7,
        )

        response = llm.invoke("请用一句话介绍什么是大语言模型RAG技术")
        print(response.content)
_set_env("GOOGLE_API_KEY", model_name="gemini-2.5-flash")


API_KEY前五位:AIzaS
大语言模型RAG技术通过**从外部知识库检索相关信息**来**增强LLM的输入上下文**，使其能够生成**更准确、更可靠、减少幻觉**的回答。


2. 扫描版/图片 PDF
PyMuPDFLoader + RapidOCR / Tesseract → 清洗 → Chunking

In [6]:
from langchain_pymupdf4llm import PyMuPDF4LLMLoader
file_path = r"knowledgeBase\yingzao_chapter01.pdf"
loader = PyMuPDF4LLMLoader(file_path) 
docs = loader.load() #返回值 docs 是一个 List[langchain.schema.Document]
docs[0]
print(type(docs))
print(type(docs[0]))


<class 'list'>
<class 'langchain_core.documents.base.Document'>


In [7]:
import pprint
pprint.pp(docs[0].page_content)
pprint.pp(docs[0].metadata)

''
{'producer': 'Microsoft: Print To PDF',
 'creator': '',
 'creationdate': '2026-01-04T21:35:34+08:00',
 'source': 'knowledgeBase\\yingzao_chapter01.pdf',
 'file_path': 'knowledgeBase\\yingzao_chapter01.pdf',
 'total_pages': 20,
 'format': 'PDF 1.7',
 'title': '《营造法式》解读 第2版 (潘谷西；何建中著) (Z-Library).pdf',
 'author': 'JunQiang Sun',
 'subject': '',
 'keywords': '',
 'moddate': '2026-01-04T21:35:34+08:00',
 'trapped': '',
 'modDate': "D:20260104213534+08'00'",
 'creationDate': "D:20260104213534+08'00'",
 'page': 0}


In [7]:
pages = []
for doc in loader.lazy_load():
    pages.append(doc)
    if len(pages) >= 10:
        # do some paged operation, e.g.
        # index.upsert(page)
        pages = []
len(pages)

0

3. 使用rapidocr

In [ ]:
from rapidocr import RapidOCR

engine = RapidOCR()

img_url = "https://github.com/RapidAI/RapidOCR/blob/main/python/tests/test_files/ch_en_num.jpg?raw=true"
result = engine(img_url)
print(result)

result.vis("vis_result.jpg")

In [ ]:
from rapidocr import EngineType, LangDet, ModelType, OCRVersion, RapidOCR

engine = RapidOCR(
    params={
        "Rec.ocr_version": OCRVersion.PPOCRV4,
        "Rec.engine_type": EngineType.ONNXRUNTIME,
        "Rec.model_type": ModelType.SERVER,
        "Det.lang_type": LangDet.CH
    }
)
doc_path = r"knowledgeBase\onePage_test.pdf"
result = engine(doc_path)
print(result)

4. 使用marker 、langchain jsonLoader

In [10]:
from langchain_core.documents import Document

document = Document(
    page_content="Hello, world!", metadata={"source": "https://example.com",
                                            "author":"sunjunqiang",
                                            "page":"1" ,
                                            "sadas":"23" ,
                                            "123":"3" }
)
print(document.page_content)

Hello, world!


In [1]:
from langchain_community.document_loaders import JSONLoader

loader = JSONLoader(
    file_path=r"knowledgeBase\output\onePage_test\onePage_test.json",
    jq_schema=".messages[].content",
    text_content=False,
)

e:\Miniconda\envs\rag\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
docs = loader.load()
docs[0]
print(docs[0].metadata)

ValueError: Cannot iterate over null (null)